# 貓狗辨識 - 建構模型
本次的課程將學習如何實作二元分類的模型，目的是利用圖片當中的特徵，來預測是貓或狗，藉由此項專案將學會如何使用python裡的套件pandas和numpy來操作資料、並利用來搭建深度學習的模型，特別是用於圖像辨識的卷積神經網絡。

### 環境提醒及備註
在執行本範例前請先確認Jupyter筆記本設置是否正確，首先點選主選單的「修改」─「筆記本設置」─「運行類別」，選擇「Python3」，同時將「硬件加速器」下拉式選單由「None」改成「GPU」，再按「保存」。

In [ ]:
### 課程架構
在貓狗辨識的專案中，將帶著學員建構一個深度學習的模型，並進行貓狗預測，主要包括下面四個步驟：

>1.   如何進行資料前處理(Processing)

>2.   如何實作資料增強(Data Augmentation)

>3.   如何建構深度學習模型(Model)

>4.   如何評估其模型的學習狀況(Inference) 

In [12]:
# 3-1
# 引入會使用到的套件

import os #處理字符串路徑
import zipfile #內建標準庫提供做壓縮與解壓縮用
import matplotlib.pyplot as plt # plt用於顯示圖片
import matplotlib.image as mpimg # mpimg 用於讀取圖片
%matplotlib inline 

import tensorflow as tf

from keras.models import Model
from keras import layers
from keras import backend as K

from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers.convolutional import Conv2D,MaxPooling2D,Convolution2D
from keras.utils.np_utils import to_categorical
import numpy as np
seed = 7
np.random.seed(seed)

### 載入資料

In [4]:
# 3-2
# 將下載到的資料集解壓縮至資料夾中

local_zip = 'cats_and_dogs_filtered.zip' # 把載下的資料帶入local_zip
zip_ref = zipfile.ZipFile(local_zip, 'r') # 讀取資料夾中的資料
zip_ref.extractall('Pics') # 解壓縮的目的路徑
zip_ref.close() # 關閉資料夾

In [5]:
# 3-3
# 取得資料集路徑

base_dir = 'Pics/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train') #連結目錄與文件名
validation_dir = os.path.join(base_dir, 'validation')

In [6]:
# 3-4
# 取得所有圖片的路徑

train_cats_dir = os.path.join(train_dir, 'cats')           # 取得訓練用貓圖片的路徑
train_dogs_dir = os.path.join(train_dir, 'dogs')           # 取得訓練用狗圖片的路徑
validation_cats_dir = os.path.join(validation_dir, 'cats') # 取得驗證用貓圖片的路徑
validation_dogs_dir = os.path.join(validation_dir, 'dogs') # 取得驗證用狗圖片的路徑

In [7]:
# 3-5
# 取得所有訓練及驗證用的圖片

train_cat_fnames = os.listdir(train_cats_dir)              # 取得訓練用所有貓圖片
train_dog_fnames = os.listdir(train_dogs_dir)              # 取得訓練用所有狗圖片
validation_cat_fnames = os.listdir(validation_cats_dir)    # 取得驗證用所有貓圖片
validation_dog_fnames = os.listdir(validation_dogs_dir)    # 取得驗用所有狗圖片

---

**3.1 卷積網路模型**

---

為了讓大家更容易了解如何利用TensorFlow及Keras建構一個深度學習模型，這裡展示了一個小型的卷積神經網路(Convolution Neural Network, CNN)，共有三層卷積層(包含ReLu及Max Pooling)，每個卷積層皆用3x3的濾波器進行卷積動作，三層分別提取16, 32及64組濾波器。接著展開成獨立節點後，再加入二層全連結層，分別為512及1個節點，而最後得到的那一個節點加上Sigmodid函數即為最終輸出的結果。其值介於0.0 ~ 1.0，當值越接近1.0時圖片為狗的機率越高，反之輸出值越接近0.0時圖片判定是貓的機率越高。

這個模型雖然不大，但可適用各種圖像分類問題，大家可試著導入自己準備的圖像進行測試。

---

**3.2 輸入圖像尺寸正規化**

---

接著說明輸入(層)資料問題。由於資料集的每張圖片大小、長寬比都不一樣，這樣無法直接進入訓練模型，因此須將影像尺寸統一縮小到指定尺寸。在這個範例中會將圖片統一縮至150x150像素，而色彩依然保持全彩(RGB 3通道)，所以對輸入層來說等於150x150x3。至於影像縮小後造成影像變形導致訓練結果有些許不同的問題是另一層次問題，在此先忽略不予考慮。

---

**3.3 模型架構及訓練參數說明**

---

1.   輸入層為150x150x3的資料，其中的3表示RGB三個通道。

2.   經過第一卷積層(Conv2D)的共產生16組3x3x3濾波器並搭配ReLu激活(Activation)函數，每一組濾波器另加上一個偏置值(Bias)一起訓練，故共有 (3x3x3+1)x16=448個參數待訓練。經過卷積後圖像大小變成148x148x16。

3.   接著以2x2窗口進行最大池化(Max Pooling)動作，取出四個值中最大值，同時將圖像縮小至一半尺寸74x74x16。這個部份不會產生待訓練參數。

4.   再來是第二卷積層(Conv2D)，這項動作類似步驟2，產生32組3x3x16濾波器，同樣搭配ReLu激活(Activation)函數，每一組濾波器另加上一個偏置值(Bias)一起訓練，故共有 (3x3x16+1)x32=4,640個參數待訓練。經過卷積後圖像大小變成72x72x32。

5.   接著同樣以2x2窗口進行最大池化(Max Pooling)動作，取出四個值中最大值，同時將圖像縮小至一半尺寸36x36x32。這個部份不會產生待訓練參數。

6.   再來進行第三卷積層(Conv2D)的共產生64組3x3x32濾波器並搭配ReLu激活(Activation)函數，每一組濾波器另加上一個偏置值(Bias)一起訓練，故共有 (3x3x32+1)x64=18,496個參數待訓練。經過卷積後圖像大小變成34x34x64。

7.   接著以2x2窗口進行最大池化(Max Pooling)動作，取出四個值中最大值，同時將圖像縮小至一半尺寸17x17x64。這個部份不會產生待訓練參數。

8.   接著將所有節點展開(Flatten)變成一維節點準備進行下一階段的全連結網路，共有17x17x64=18,496個節點。

9.   再來以512個節點和上一層進行全連結(Dense)，並搭配ReLu激活函數，再加上一個偏置值(Bias)一起訓練，故共有(18,496+1)x512=9,470,464個參數待訓練。

10.  最後只留一個輸出節點和上一層進行全連結(Dense)並搭配Sigmoid激活函數來產生最後輸出結果，加上一個偏置值(Bias)，故共有512+1=513個參數待訓練。

11.  合計共有9,494,561個參數待訓練。

In [10]:
# 3-6

# Configure the TF backend session
tf_config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(allow_growth=True))
K.set_session(tf.Session(config=tf_config))

# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)

# 展示模型結構及各層所需訓練參數
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 64)        0         
__________

 ---

**3.4 模型配置及訓練優化設定**

---

接下來配置模型訓練的規格。這裡用binary_crossentropy損失來訓練模型，因為這個範例是一個二元分類問題，而最終激活函數是sigmoid。 而優化器部份使用rmsprop自動學習速率調速，學習速率初始為0.001。 在訓練期間，需監控分類的準確性。

補充說明一下，使用RMSprop優化算法優於隨機梯度下降（SGD），因為RMSprop為我們實現了自動學習速率調整。而其它如Adam和Adagrad等優化器也會在訓練期間自動調整學習速率，若使用在本範例亦可以很好地工作。

In [11]:
# 3-7

from keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [13]:
# 3-8

model = Sequential()
#第一層
#Conv2D以 2的四次方 16組 3X3 kernal 
#strides每次移動步伐是1X1
#input_shape輸入圖像的格式是150X150X3
#activation使用的激活函數
model.add(Conv2D(16,(3,3),strides=(1,1),input_shape=(150, 150, 3),padding='valid',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None))

#第二層
#Conv2D以2的五次方 32組 3X3 kernal 
#strides每次移動步伐是1X1
#input_shape輸入圖像的格式是150X150X3
#activation使用的激活函數
model.add(Conv2D(32,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None))

#第三層
#Conv2D以2的六次方 64組 3X3 kernal 
#strides每次移動步伐是1X1
#input_shape輸入圖像的格式是150X150X3
#activation使用的激活函數
model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None))

#第四層
#平坦化
model.add(Flatten())

#第五層
#全連接層 512 1X1 kernal 
model.add(Dense(512,activation='relu'))

#第五層
#全連接層 512 1X1 kernal
model.add(Dense(1, activation='sigmoid'))
          
# model.add(Dropout(0.5))
# model.add(Dense(4096,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1000,activation='softmax'))
# model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 74, 74, 32)        4640      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 20736)             0         
__________

---